# 오늘의 영어 단어 생성하기

In [1]:
import json

from tqdm.notebook import tqdm
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain.pydantic_v1 import BaseModel, Field
from langchain.schema import HumanMessage, AIMessage
import pandas as pd #엑셀파일로 저장하기 위해 필요한 라이브러리

C:\Users\msh07\miniconda3\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=1.0) #단어를 랜덤 확률을 높이기 위해 temperature를 1로 높게 설정.

In [43]:
class Word(BaseModel):
    word: str = Field("영어 단어")
    imoj: str = Field("word에 어울리는 적절한 이모지")
    meaning: str = Field("word의 의미, 뜻 한국어로")
    origin: str = Field("word의 어원, 100자 이내 한국어로")
    synonym: str = Field("word와 유사한 뜻을 가진 영단어")

In [44]:
parser = JsonOutputParser(pydantic_object=Word) #parser 노드!

In [45]:
format_instruction = parser.get_format_instructions()

In [46]:
#fewshot : 원하는 답변의 예시를 미리 제시함. 답변을 강화하기 위해 반복적으로 예를 제시. 반복되는 예시에 패턴을 읽어서 예시와 유사하게 답변을 제시함. format_instruction에서도 동일하지만 fewshot이 더 나음.
fewshot = [
    HumanMessage(content=f"영어 단어 하나와 그 단어에 어울리는 이모지와 그 뜻을 만들어줘. 포맷은 다음을 따라 만들어줘\n{format_instruction}"),
    AIMessage(content='{"word": "situation", "imoj": "🤔", "meaning": "situation은 어떤 일이 일어나고 있는 특정한 장소나 시간을 말합니다. 또한 그것은 어떤 것의 현재 상태나 조건을 가리킬 수도 있습니다. situation은 문제나 어려움이 발생한 상황을 강조하기 위해 사용될 수 있습니다.", "origin": ""situation"은 라틴어 situatio ( situare, "배치하다")에서 유래했습니다. 이후 고대 프랑스어 situacion을 거쳐 중세 영어로 들어왔으며, 물리적 위치에서 상황이나 환경을 의미하는 단어로 발전했습니다.","synonym": "circumstance"}'),
    HumanMessage(content=f"영어 단어 하나와 그 단어에 어울리는 이모지와 그 뜻을 만들어줘. 포맷은 다음을 따라 만들어줘\n{format_instruction}"),
    AIMessage(content='{"word": "concern", "imoj": "😟", "meaning": "concern은 걱정이나 근심을 의미합니다. 그것은 누군가에게 감정적으로나 육체적으로 영향을 미칠 수 있는 무언가에 대한 관심이나 걱정을 나타낼 수 있습니다. concern은 또한 중요성이나 관련성을 의미할 수도 있습니다.", "orogin": ""concern"은 라틴어 concernere ("함께 섞다, 체로 치다")에서 유래했으며, con- ("함께")과 cernere ("체로 치다, 분리하다")에서 파생되었습니다. 이후 고대 프랑스어 concerner ("관련되다")를 거쳐 15세기 중세 영어로 들어와 걱정이나 관심을 나타내는 의미로 발전했습니다.", "synonym": "worry"}'),
    HumanMessage(content=f"영어 단어 하나와 그 단어에 어울리는 이모지와 그 뜻을 만들어줘. 포맷은 다음을 따라 만들어줘\n{format_instruction}"),
]

In [47]:
chain = model | parser

In [48]:
chain.invoke(fewshot)

{'word': 'serendipity',
 'imoj': '🍀',
 'meaning': 'Serendipity는 우연히 발견한 행운이나 기쁨을 의미합니다. 어떤 일이 계획되지 않은 상태에서 예기치 않게 발생하는 긍정적인 사건이나 경험을 가리킬 때 사용됩니다.',
 'origin': 'Serendipity는 18세기 영국의 작가인 홀로우드 잡스가 처음 사용한 용어로, 이탈리아 명칭 Serendip (과거의 스리랑카 섬 이름)에 -ity 접미사를 붙여 만들어졌습니다. Serendip은 다른 것을 찾아나서 이외의 도람을 일으키는 사람이라는 뜻을 가지고 있습니다.',
 'synonym': 'chance finding'}

In [9]:
sample_list = []
n_sample = 10
for _ in tqdm(range(n_sample), total=n_sample):
    sample = chain.invoke(fewshot)
    sample_list.append(sample)
    

In [10]:
df = pd.DataFrame(sample_list)

In [11]:
df.to_excel("./words.xlsx", index=False)